In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [ ]:
ords_prods_merge = pd.read_pickle(r'/Users/jlsanabria77/Desktop/17-02.2021 Instacat Basket Analysis 3/02 Data/Prepared Data/ords_prods_merge.pkl')

In [3]:
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


3. When analyzing the results there are few aspects worth to mention. First the enire dataframe produces 21 departments while the subset only showed eight. Also When looking at the mean from each deapartment we can see that those are slightly lower. For example, department_id 7 has a mean of 17.472355 in the subset but a mean of 17.225802 in the entire dataframe.

2. Create a Loyalty flag

In [4]:
# 1st. I will split the data into groups based on the “user_id” column.
# 2nd. I will apply the transform() function on the “order_number” column to generate the maximum orders for each user.
# 3rd. I will create a new column, “max_order,” into which I will place the results of the aggregation.

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [6]:
# Check the new column has been created
ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average Orders,10
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Soda,77,7,9.0,both,Mid-range product,Slowest day,Slowest days,Average Orders,10
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Soda,77,7,9.0,both,Mid-range product,Slowest day,Slowest days,Most Orders,10
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Soda,77,7,9.0,both,Mid-range product,Slowest day,Slowest days,Average Orders,10
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Soda,77,7,9.0,both,Mid-range product,Slowest day,Slowest days,Most Orders,10


Create a flag that assigns a “loyalty” label to a user ID based on its corresponding max order value

In [ ]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [8]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [9]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [10]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

5. Determine whether prices of products purchased by loyal customers differ from those purchased by regular or new customers

In [11]:
ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average Orders,10,New customer
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,77,7,9.0,both,Mid-range product,Slowest day,Slowest days,Average Orders,10,New customer
2,473747,1,prior,3,3,12,21.0,196,1,1,...,77,7,9.0,both,Mid-range product,Slowest day,Slowest days,Most Orders,10,New customer
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,77,7,9.0,both,Mid-range product,Slowest day,Slowest days,Average Orders,10,New customer
4,431534,1,prior,5,4,15,28.0,196,1,1,...,77,7,9.0,both,Mid-range product,Slowest day,Slowest days,Most Orders,10,New customer


In [12]:
# By using the groupby I will do multiple aggregations to find out the maximum, minimum and average price of products bought by these three type of customers
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean','max','min']})

prices              
                       mean      max  min
loyalty_flag                             
Loyal customer    10.386336  99999.0  1.0
New customer      13.294670  99999.0  1.0
Regular customer  12.495717  99999.0  1.0

We can see that all three groups buy from the cheapest (1.0) to the most expensive (99999.0) product. 
Also we observe that Loyal Customers buy on average less expensive products than the other two groups. 

6. Create a spending flag for each user based on the average price across all their orders using the following criteria:

In [13]:
# 1st. I will create a column of mean spend by user_id
ords_prods_merge['mean_spend'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [ ]:
# 2nd. I will create a spend flag for "Low Spender"
ords_prods_merge.loc[ords_prods_merge['mean_spend'] < 10, 'spend_flag'] = 'Low Spender'

In [ ]:
# 3rd. I will create a spend flag for "High Spender"
ords_prods_merge.loc[ords_prods_merge['mean_spend'] >= 10, 'spend_flag'] = 'High Spender'

In [ ]:
# 4th. I will check all values 
ords_prods_merge['spend_flag'].value_counts(dropna = False)

7. Determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column.

In [ ]:
# 1st. I will create a column of median for days_since_prior_order column
ords_prods_merge['median_ordering'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [ ]:
# 2nd. I will create a frequency flag for "Non-frequent-customer"
ords_prods_merge.loc[ords_prods_merge['median_ordering'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [ ]:
# 3th. I will create a frequency flag for "Regular customer"
ords_prods_merge.loc[(ords_prods_merge['median_ordering'] > 10) & (ords_prods_merge['median_ordering'] <= 20), 'frequency_flag'] = 'Regular customer'

In [ ]:
# 4th. I will create a frequenct flag for "Frequent customer"
ords_prods_merge.loc[ords_prods_merge['median_ordering'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [ ]:
# 5th. I will check all values
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

In [ ]:
# Export dataframe as pikle file to "Prepared Data" folder
ords_prods_merge.to_pickle(os.path.join(r'/Users/jlsanabria77/Desktop/17-02.2021 Instacat Basket Analysis 3/02 Data/Prepared Data/ords_prods_merge.pkl'))